In [1]:
import pandas as pd
import subprocess
import glob
import numpy as np

In [3]:
# prior to this, train final model on 100% of data using script_2A_gnn_final_train.sh

# Part 1: Use final model on Broad800k library

In [ ]:
# use a small chunk of 1,000 compounds at a time for models to predict on
# can do less or more depending on your RAM
chunksize = 1000

# make directory for results to live
!mkdir broad800k_preds/

# preprocess data - SMILES and RDKit 2D normalized features
data = pd.read_csv('../data/broad800k.csv')
with np.load('../data/broad.npz') as ftsdata:
    ftsdata = ftsdata['features']

# chunkify the features and SMILES and save the pre-chunked features to files
for i in range(0, len(data), chunksize):
    smis = pd.DataFrame(data.iloc[i:i+chunksize,:])
    smis.to_csv('broad800k_preds/' + str(i) + '.csv', index = False)
    nps = ftsdata[i:i+chunksize,:]
    np.save('broad800k_preds/' + str(i) + '.npy', nps)

# to predict, use subprocess, parallel, or whatever you want!
for i in range(0, len(data), chunksize):
    print(i)
    activate_command = 'conda activate chemprop; '
    run_command = 'python predict.py --test_path ../nontoxic_stat_phase_killing_abx/broad800k_preds/' + str(i) + '.csv --checkpoint_dir ../nontoxic_stat_phase_killing_abx/models/finaltrain16/ --preds_path ../nontoxic_stat_phase_killing_abx/broad800k_preds/' + str(i) + '_preds.csv --features_path ../nontoxic_stat_phase_killing_abx/broad800k_preds/' + str(i) + '.npy --no_features_scaling --smiles_column smiles --gpu 0 --ensemble_variance'
    full_command = activate_command + run_command
    test = subprocess.run(full_command, cwd="../chemprop/", shell=True, capture_output=True)

360000
361000
362000
363000
364000
365000
366000
367000
368000
369000
370000
371000
372000
373000
374000
375000
376000
377000
378000
379000
380000
381000
382000
383000
384000
385000
386000
387000
388000
389000
390000
391000
392000
393000
394000
395000
396000
397000
398000
399000
400000
401000
402000
403000
404000
405000
406000
407000
408000
409000
410000
411000
412000
413000
414000
415000
416000
417000
418000
419000
420000
421000
422000
423000
424000
425000
426000
427000
428000
429000
430000
431000
432000
433000
434000
435000
436000
437000
438000
439000
440000
441000
442000
443000
444000
445000
446000
447000
448000
449000
450000
451000
452000
453000
454000
455000
456000
457000
458000
459000
460000
461000
462000
463000
464000
465000
466000
467000
468000
469000
470000
471000
472000
473000
474000
475000
476000
477000
478000
479000
480000
481000
482000
483000
484000
485000
486000
487000
488000
489000
490000
491000
492000
493000
494000
495000
496000
497000
498000
499000
500000
501000
502000

In [ ]:
# now combine predictions across all chunks
chemprop = pd.read_csv('broad800k_preds/0_preds.csv')
for i in range(chunksize, len(data), chunksize):
    new = pd.read_csv('broad800k_preds/' + str(i) + '_preds.csv')
    chemprop = pd.concat([chemprop, new])
chemprop.to_csv('../out/broad800k_model_predictions_08_20_2021.csv', index = False)